# Basics of Long Short Term Memory 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#Requirements" data-toc-modified-id="Requirements-2">Requirements</a></span><ul class="toc-item"><li><span><a href="#Knowledge" data-toc-modified-id="Knowledge-2.1">Knowledge</a></span></li><li><span><a href="#Python-Modules" data-toc-modified-id="Python-Modules-2.2">Python Modules</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-2.3">Data</a></span></li></ul></li><li><span><a href="#Recap:-Recurrent-Neural-Network" data-toc-modified-id="Recap:-Recurrent-Neural-Network-3">Recap: Recurrent Neural Network</a></span></li><li><span><a href="#Long-Short-Term-Memory" data-toc-modified-id="Long-Short-Term-Memory-4">Long Short Term Memory</a></span><ul class="toc-item"><li><span><a href="#Implementation" data-toc-modified-id="Implementation-4.1">Implementation</a></span><ul class="toc-item"><li><span><a href="#Functions-for-the-LSTM" data-toc-modified-id="Functions-for-the-LSTM-4.1.1">Functions for the LSTM</a></span></li><li><span><a href="#Forward-pass" data-toc-modified-id="Forward-pass-4.1.2">Forward pass</a></span></li><li><span><a href="#Backward-path" data-toc-modified-id="Backward-path-4.1.3">Backward path</a></span></li><li><span><a href="#Sampling" data-toc-modified-id="Sampling-4.1.4">Sampling</a></span></li><li><span><a href="#Trainings-process" data-toc-modified-id="Trainings-process-4.1.5">Trainings process</a></span><ul class="toc-item"><li><span><a href="#Hyperparameter" data-toc-modified-id="Hyperparameter-4.1.5.1">Hyperparameter</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4.1.5.2">Main</a></span></li></ul></li><li><span><a href="#Learning-curve" data-toc-modified-id="Learning-curve-4.1.6">Learning curve</a></span></li></ul></li></ul></li><li><span><a href="#Licenses" data-toc-modified-id="Licenses-5">Licenses</a></span><ul class="toc-item"><li><span><a href="#Notebook-License-(CC-BY-SA-4.0)" data-toc-modified-id="Notebook-License-(CC-BY-SA-4.0)-5.1">Notebook License (CC-BY-SA 4.0)</a></span></li><li><span><a href="#Code-License-(MIT)" data-toc-modified-id="Code-License-(MIT)-5.2">Code License (MIT)</a></span></li></ul></li></ul></div>

## Introduction

TODO

## Requirements

### Knowledge

- [Recommended] [Neural Network and Deep Learning - Backpropagation](http://neuralnetworksanddeeplearning.com/chap2.html)
- [Recommended] [Matrix Calculus](https://explained.ai/matrix-calculus/index.html)
- [Recommended] [cs231 Recurrent Neural Network](https://www.youtube.com/watch?v=6niqTuYFZLQ)
- [Recommended] [Basics of Recurrent Neural Network Notebook]

### Python Modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

### Data

In [ ]:
data = open('data.txt', 'r').read()

text_length = len(text)
chars = list(set(text))
char_length = len(chars)

# dictionaries which we will use in the future for transformations
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# transform the text string to a list of integers
X = np.array([char_to_int[char] for char in text])

print('text:\n', text[:200], '\n')
print('length of text:\n', text_length, '\namount of characters:\n', char_length)
print('alphabet:\n', chars,'\n')
print('first 10 datas:\n', X[0:10])

## Recap: Recurrent Neural Network

**[[Summary from colah's blog]](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)** Humans don’t start their thinking from scratch every second. As you read this, you understand each word based on your understanding of previous words. You don’t throw everything away and start thinking from scratch again. Your thoughts have persistence. For example, imagine you want to classify what kind of event is happening at every point in a movie. It’s unclear how a traditional neural network could use its reasoning about previous events in the film to inform later ones.

Recurrent neural networks address this issue. They are networks with loops in them, allowing information to persist. The idea is, that they might be able to connect previous information to the present task, such as using previous video frames might inform the understanding of the present frame. 

But in theory, RNNs are absolutely capable of handling “long-term dependencies". In practice, RNNs don’t seem to be able to learn them. The problem was explored in depth by Hochreiter (1991) and Bengio, et al. (1994), who found some pretty fundamental reasons why it might be difficult.

## Long Short Term Memory
Long Short Term Memory networks are Recurrent Neural Networks capable of learning long-term dependencies. They were introduced by Hochreiter & Schmidhuber (1997). They work tremendously well on a large variety of problems, and are now widely used.

LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is their default behavior, not something they struggle to learn! Lets have a look at a single LSTM cell:

<img src="images/lstm-diagram.jpg" width="400px">

- $f$ forget gate, whether to erase the cell
- $i$ input gate, whether to write to cell
- $g$ gate gate, how much to write to a cell
- $o$ output gate, how much to reveal a cell
- $c_t$ cell state at a time $t$
- $h_t$ hidden state at a time $t$
- $y_t$ output at a time $t$ 

### Implementation
Here we go over the core functions of a recurrent neural network. Every function could be added to a ```lstm.py``` script to run the long short term memory network outside of a notebook. This implementation is not efficient though and it is not recommended to use it on a very large text.

#### Functions for the LSTM
The long short term memory uses the sigmoid and the tanh function. Instead of hard-coding the functions into the forward method, we use helper/utility methods for a much cleaner code. You can imagine these snippets inside a ```utility.py``` or as private methods of ```lstm.py```.

$$
\begin{aligned}
\sigma(x) = \frac{1}{1 + e^{-x}} && \sigma^\prime(x) = \sigma(x) (1 - \sigma(x))
\end{aligned}
$$

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dsigmoid(x):
    out = 1 / (1 + np.exp(-x))
    return out * (1 - out)

$$
\begin{aligned}
\tanh(x) && \tanh^\prime(x) = 1 - \tanh^2(x)
\end{aligned}
$$

In [ ]:
def tanh(x):
    return np.tanh(x)

def dtanh(x):
    return 1 + np.tanh(x) ** 2

#### Forward pass

$$
\begin{aligned} \left( \begin{array} { c } { i } \\{ f } \\ { o } \\ { g } \end{array} \right) & = \left( \begin{array} { c } { \sigma } \\ { \sigma } \\ { \sigma } \\ { \tanh } \end{array} \right) W \left( \begin{array} { c } { h _ { t - 1 } } \\ { x _ { t } } \end{array} \right) \\ c _ { t } & = f \odot c _ { t - 1 } + i \odot g \\ h _ { t } & = o \odot \tanh \left( c _ { t } \right) \\
y_t &= W_{y} * h_t \\
p_t &=\frac{e^{y_t}}{\sum_k e^{y_k}} \\
L &= \sum_t - log (p_t)\end{aligned}
$$

In [ ]:
hidden_size = 100
seq_size = 25
stack_size = hidden_size + char_length

In [ ]:
# weights of the gates
Wf = np.random.randn(hidden_size, stack_size) * 0.01
Wi = np.random.randn(hidden_size, stack_size) * 0.01
Wg = np.random.randn(hidden_size, stack_size) * 0.01
Wo = np.random.randn(hidden_size, stack_size) * 0.01

# bias of the gates
bf = np.zeros((hidden_size, 1)) * 0.01
bi = np.zeros((hidden_size, 1)) * 0.01
bg = np.zeros((hidden_size, 1)) * 0.01
bo = np.zeros((hidden_size, 1)) * 0.01

Wy = np.random.randn(char_length, hidden_size) * 0.01
by = np.zeros((char_length, 1)) * 0.01

In [ ]:
def forward(X, Y, hprev, cprev):
    # initializing our variables
    h, c, p, loss = [hprev], [cprev], [], 0
    f, i, g, o = [], [], [], []
    for t in range(len(X)):
        # transforming the a datapoint at a time step to a one hot encoded vector
        xt = np.zeros((char_length, 1))
        xt[X[t]] = 1
        # stack the hidden state and data point together
        stacked = np.append(h[t], xt)
        stacked = stacked.reshape(stacked.shape[0], 1)
        # get the 4 gates described at the diagram
        f.append(sigmoid(Wf @ stacked + bf))
        i.append(sigmoid(Wi @ stacked + bi))
        g.append(sigmoid(Wo @ stacked + bg))
        o.append(tanh(Wg @ stacked + bo))
        # calculate the new cell and hidden state
        c.append(c[t] * f[t] + i[t] * g[t])
        h.append(tanh(c[t + 1]) * o[t])
        # calculate the out/prediction for the input data based on the current hidden state
        y = np.dot(Wy, h[t + 1])
        p.append(np.exp(y) / np.sum(np.exp(y)))
        # calculate loss of the prediction
        loss += -np.sum(np.log(p[t][Y[t], 0]))
    return h, c, f, i, g, o, p, loss


#### Backward path
$$
\begin{aligned}
\frac{\partial L_t}{\partial W_{hy}} = (p_t - label_t) *h_t^T  \\ \frac{\partial L_t}{\partial h_t} = (p_t - label_t) * W_{hy}^T\\
\frac{\partial h_t}{\partial c_t} = o \odot \tanh^\prime(c_t) & \quad \frac{\partial h_t}{\partial o_t} = \tanh(c_t)
\\
\frac{\partial c_t}{\partial f_t} = c_{t-1} \quad \frac{\partial c_t}{\partial g_t} = i  \quad \frac{\partial c_t}{\partial i_t} = g &
\\
\frac{\partial f_t}{\partial W} =\sigma^\prime \text{stack}_t^T \quad \frac{\partial i_t}{W} = \sigma^\prime \text{stack}_t^T\quad \frac{\partial g_t}{\partial W} = \tanh^\prime \text{stack}_t^T &\quad \frac{\partial o_t}{\partial W} = \sigma^\prime \text{stack}_t^T\\
\frac{\partial h_t}{\partial h_{t-1}}=  &\quad \frac{\partial c_t}{\partial c_{t-1}} = 
\end{aligned}
$$

In [ ]:
def backward(X, Y, h, c, f, i, g, o, p):
    # initializes the gradients with zeros
    dWf, dWi, dWo, dWg, dWy = np.zeros_like(Wf), np.zeros_like(
        Wi), np.zeros_like(Wo), np.zeros_like(Wg), np.zeros_like(Wy)
    dbf, dbi, dbg, dbo, dby = np.zeros_like(bf), np.zeros_like(
        bi), np.zeros_like(bg), np.zeros_like(bo), np.zeros_like(by)

    dhprevious = np.zeros_like(h[0])
    dcprevious = np.zeros_like(c[0])
    for t in reversed(range(len(X))):
        # transforming the a datapoint at a time step to a one hot encoded vector
        xt = np.zeros((char_length, 1))
        xt[X[t]] = 1
        # stack the hidden state and data point together
        stacked = np.append(h[t], xt)
        stacked = stacked.reshape(stacked.shape[0], 1)
        # gradient of Why
        dy = np.copy(p[t])
        dy[Y[t]] -= 1
        dby += dy
        dWy += np.dot(dy, h[t].T)
        # gradient of Wf, Wi, Wg, Wo
        dh = np.dot(Wy.T, dy) + dhprevious  # backprop into h
        # derivative of the hidden state with respect to the cell state 
        dc = o[t] * dtanh(c[t]) * dh + dcprevious
        # derivative of the hidden state with respect to the output cate
        do = dh * tanh(c[t]) * dsigmoid(o[t])
        # derivative of the cell state with respect to the gates
        df = dc * c[t-1] * dsigmoid(f[t])
        di = dc * g[t] * dsigmoid(i[t])
        dg = dc * i[t]
        dg = dg * dtanh(dg)
        # derivative of the gates with respect to the weights
        dbf += df
        dWf += np.dot(df, stacked.T) 
        dbi += di
        dWi += np.dot(di, stacked.T)
        dbg += dg
        dWg += np.dot(dg, stacked.T)
        dbo += do
        dWo += np.dot(do, stacked.T)
        # gradient flow
        dhprevious = np.dot(Wf.T, df) + np.dot(Wi.T, di) + \
            np.dot(Wo.T, dc) + np.dot(Wg.T, dg)
        dhprevious = dhprevious[:hidden_size, :]
        dcprevious = dc * f[t]

    # gradient clip to mitigate exploding gradients
    for dparam in [dWf, dWi, dWg, dWo, dWy, dbf, dbi, dbg, dbo, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    return dWf, dWi, dWg, dWo, dWy, dbf, dbi, dbg, dbo, dby

#### Sampling

In [ ]:
def sample(start_char, h, c, n):
    '''
    This functions returns a sentence of length n based on the starting character, the latest hidden state.

    Args:
        start_char: the character which we start with as integer
        h: latest hidden state
        n: length of sentence as integer

    Returns:
        A sample sentence as String
    '''
    # transforming the a starting character to a one hot encoded vector
    xt = np.zeros((char_length, 1))
    xt[start_char] = 1
    # initializing output
    text = ''
    for t in range(n):
        # stack the hidden state and data point together
        stacked = np.append(h, xt)
        stacked = stacked.reshape(stacked.shape[0], 1)
        # get the 4 gates described at the diagram
        f = sigmoid(Wf @ stacked + bf)
        i = sigmoid(Wi @ stacked + bi)
        g = sigmoid(Wo @ stacked + bg)
        o = tanh(Wg @ stacked + bo)
        # calculate the new cell and hidden state
        c = (c * f + i * g)
        h = tanh(c) * o
        # calculate the out/prediction for the input data based on the current hidden state
        y = np.dot(Wy, h)
        p = np.exp(y) / np.sum(np.exp(y))
        # adds the predicted character to our output String
        text += chars[np.argmax(p)]
        # generates a random sample/new character from a given array based on a probability distributipn p
        random_index = np.random.choice(range(char_length), p=p.ravel())
        # transforming the generated character to a one hot encoded vector
        xt = np.zeros((char_length, 1))
        xt[random_index] = 1
    return text

#### Trainings process

##### Hyperparameter

In [ ]:
# learning rate for the gradient descent algorithm
learning_rate = 1e-1

# how many times the model sees the whole data
iterations = 1000

##### Main

In [ ]:
grad_squared_Wf, grad_squared_Wi, grad_squared_Wg, grad_squared_Wo, grad_squared_Wy = np.zeros_like(
    Wf), np.zeros_like(Wi), np.zeros_like(Wg), np.zeros_like(Wo), np.zeros_like(Wy)

grad_squared_bf, grad_squared_bi, grad_squared_bg, grad_sqared_bo, grad_squared_by = np.zeros_like(bf), np.zeros_like(bi), np.zeros_like(bg), np.zeros_like(bo), np.zeros_like(by)

#### Learning curve

In [ ]:
# initializing hidden state, loss history, squared gradient and loss
loss_history = []
h, c = 0, 0
loss = 0
steps = 0
smooth_loss = -np.log(1.0 / text_length) * seq_size

for iteration in tqdm(range(iterations)):
    if steps+seq_size+1 >= text_length or iteration == 0:
        h = [np.zeros((hidden_size, 1))]
        c = [np.zeros((hidden_size, 1))]
        steps = 0
    # splits the data to the right sequence size
    inputs = X[steps:steps+seq_size]
    targets = X[steps+1:steps+1+seq_size]
    # calculate the new hidden state, probability distribution and loss
    h, c, f, i, g, o, p, loss = forward(inputs, targets, h[-1], c[-1])
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    loss_history.append(smooth_loss)
    # get the gradients
    dWf, dWi, dWg, dWo, dWy, dbf, dbi, dbg, dbo, dby = backward(inputs, targets, h, c, f, i, g, o, p)
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wf, Wi, Wg, Wo, Wy, bf, bi, bg, bo, by],
                                  [dWf, dWi, dWg, dWo, dWy, dbf, dbi, dbg, dbo, dby],
                                  [grad_squared_Wf, grad_squared_Wi, grad_squared_Wg, grad_squared_Wo, grad_squared_Wy, grad_squared_bf, grad_squared_bi, grad_squared_bg, grad_sqared_bo, grad_squared_by]):
        mem += dparam * dparam
        param += -learning_rate * dparam #/ \
            #np.sqrt(mem + 1e-8)  # adagrad update

    if iteration % 10 == 0:
        print('sample at iteration:', iteration, 'with loss of:', smooth_loss)
        print(sample(inputs[0], h[-1], c[-1], 150), '\n')

    steps += seq_size

In [ ]:
plt.plot(loss_history)
plt.ylabel('loss')
plt.xlabel('iterations')

## Licenses

### Notebook License (CC-BY-SA 4.0)

*The following license applies to the complete notebook, including code cells. It does however not apply to any referenced external media (e.g., images).*

_Notebook title_ <br/>
by _Author (provide a link if possible)_ <br/>
is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/).<br/>
Based on a work at https://gitlab.com/deep.TEACHING.


### Code License (MIT)

*The following license only applies to code cells of the notebook.*

Copyright 2018 Benjamin Voigt, Steven Mi

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.